## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 

city_data_df = pd.read_csv("cities.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Ushuaia,AR,-54.8000,-68.3000,31.82,93,100,6.91
1,1,Mahajanga,MG,-15.7167,46.3167,75.22,57,40,11.50
2,2,Busselton,AU,-33.6500,115.3333,57.25,74,95,7.65
3,3,Kununurra,AU,-15.7667,128.7333,48.18,87,9,2.30
4,4,Sawtell,AU,-30.3833,153.1000,51.82,66,19,10.36


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Mahajanga,MG,-15.7167,46.3167,75.22,57,40,11.50
7,7,Fomboni,KM,-12.2800,43.7425,76.39,71,7,13.11
8,8,Jalu,LY,29.0331,21.5482,87.48,23,0,9.35
14,14,Torbay,CA,47.6666,-52.7314,75.97,45,40,17.27
18,18,Along,IN,28.1667,94.7667,75.97,98,67,2.06
21,21,Hithadhoo,MV,-0.6000,73.0833,83.08,78,57,15.21
29,29,Kanigiri,IN,15.4000,79.5167,82.29,69,100,17.11
33,33,Rongcheng,CN,30.6340,117.8517,80.98,86,100,3.56
35,35,Vaini,TO,-21.2000,-175.2000,75.36,94,20,5.75
36,36,Jamestown,US,42.0970,-79.2353,81.95,43,58,1.30


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       705
City          705
Country       705
Lat           705
Lng           705
Max Temp      705
Humidity      705
Cloudiness    705
Wind Speed    705
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Mahajanga,MG,-15.7167,46.3167,75.22,57,40,11.50
7,7,Fomboni,KM,-12.2800,43.7425,76.39,71,7,13.11
8,8,Jalu,LY,29.0331,21.5482,87.48,23,0,9.35
14,14,Torbay,CA,47.6666,-52.7314,75.97,45,40,17.27
18,18,Along,IN,28.1667,94.7667,75.97,98,67,2.06
...,...,...,...,...,...,...,...,...,...
1820,1820,Montrose,US,38.4166,-108.2673,83.61,25,40,9.13
1828,1828,Middlebury,US,44.0153,-73.1673,84.00,54,75,0.00
1829,1829,Puerto Madero,MX,14.7208,-92.4215,84.69,72,99,8.72
1831,1831,Manzanillo,CU,20.3433,-77.1167,85.60,66,84,8.48


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Mahajanga,MG,75.22,-15.7167,46.3167,
7,Fomboni,KM,76.39,-12.2800,43.7425,
8,Jalu,LY,87.48,29.0331,21.5482,
14,Torbay,CA,75.97,47.6666,-52.7314,
18,Along,IN,75.97,28.1667,94.7667,
21,Hithadhoo,MV,83.08,-0.6000,73.0833,
29,Kanigiri,IN,82.29,15.4000,79.5167,
33,Rongcheng,CN,80.98,30.6340,117.8517,
35,Vaini,TO,75.36,-21.2000,-175.2000,
36,Jamestown,US,81.95,42.0970,-79.2353,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
   # 7. Drop the rows where there is no Hotel Name.


    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):  
        
        print("Hotel not found... skipping.")  
       
  

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 8a. Create the output File (CSV)
output_data_file = "cities.csv"

# 8b. Export the City_Data into a csv
preferred_cities_df.to_csv(output_data_file, index_label="City_ID")

In [10]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = preferred_cities_df[["Lat", "Lng"]]

In [ ]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [11]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig


Figure(layout=FigureLayout(height='420px'))